<a href="https://colab.research.google.com/github/Djensonsan/Spotify-Sequential-Skip-Prediction-Challenge/blob/main/similarity_measures/cosine_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/Djensonsan/Spotify-Sequential-Skip-Prediction-Challenge/blob/main/similarity_measures/mahalanobis_distance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cosine Similarity
By Jens Leysen


## Imports & Constants

In [1]:
# Install your required packages here
!pip install pandas numpy matplotlib sklearn fsspec gcsfs tqdm

     |████████████████████████████████| 92kB 3.8MB/s 
     |████████████████████████████████| 1.3MB 7.2MB/s 
     |████████████████████████████████| 296kB 16.2MB/s 
     |████████████████████████████████| 143kB 10.6MB/s 
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-cp36-none-any.whl size=3161 sha256=1c221026dbc3a276c2c6f1bba112aadf6db0b2d79998a4f07970a5c4755e349b
  Stored in directory: /root/.cache/pip/wheels/d3/00/b3/32d613e19e08a739751dd6bf998cfed277728f8b2127ad4eb7
Successfully built idna-ssl


In [2]:
# Path to credentials for cloud bucket:
%env GOOGLE_APPLICATION_CREDENTIALS=/content/drive/My Drive/CS/AI/Credentials/ai-project-2020-f4dfbc25326c.json

env: GOOGLE_APPLICATION_CREDENTIALS=/content/drive/My Drive/CS/AI/Credentials/ai-project-2020-f4dfbc25326c.json


In [42]:
from google.cloud import storage

import numpy as np
import time
import pandas as pd
import sklearn
from glob import glob
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
pd.set_option('display.max_rows', 800)
pd.set_option('display.max_columns', 800)

In [52]:
from tqdm import tqdm

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# define constants
bucket_name = "ai-project-2020-spotify"
client = storage.Client()
bucket = client.get_bucket(bucket_name)

## Import Session Logs

In [8]:
# Cloud bucket contains larger datasets:
train_files = list(bucket.list_blobs(prefix='training_set/'))
for blob in [blob for blob in train_files if '20180715' in blob.name]:
  print(blob.name)

training_set/log_0_20180715_000000000000.csv.gz
training_set/log_1_20180715_000000000000.csv.gz
training_set/log_2_20180715_000000000000.csv.gz
training_set/log_3_20180715_000000000000.csv.gz
training_set/log_4_20180715_000000000000.csv.gz
training_set/log_5_20180715_000000000000.csv.gz
training_set/log_6_20180715_000000000000.csv.gz
training_set/log_7_20180715_000000000000.csv.gz


In [9]:
#Cloud bucket contains larger datasets:
logs = pd.read_csv(f"gs://{bucket_name}/training_set/log_0_20180715_000000000000.csv.gz")
# Path to mini logs:
# logs = pd.read_csv('/content/drive/My Drive/CS/AI/Data/mini/log_mini.csv')
# logs.columns

In [10]:
def categorical_to_dummies(df, categorical_cols):
    """ Create dummies (one hot encoding) for each categorical variables """
    dummies = pd.get_dummies(df[categorical_cols], prefix=categorical_cols)
    return df.drop(columns=categorical_cols).join(dummies)

In [11]:
# Remove date for convenience (could encode this as well)
logs.drop(columns=['date'], inplace=True)

# Create dummies (one hot encoding) for each categorical variable in logs
categorical_cols = ['context_type', 'hist_user_behavior_reason_start', 'hist_user_behavior_reason_end']
logs = categorical_to_dummies(logs, categorical_cols)
print(logs.shape)

(2990609, 44)


## Import Track Features

In [12]:
track_features_1 = pd.read_csv('/content/drive/My Drive/CS/AI/Data/track_features/tf_000000000000.csv').set_index('track_id')
track_features_2 = pd.read_csv('/content/drive/My Drive/CS/AI/Data/track_features/tf_000000000001.csv').set_index('track_id')
track_features = track_features_1.append(track_features_2)

In [13]:
# Create dummies (one hot encoding) for each categorical variable in track_features
track_features = categorical_to_dummies(track_features, ['mode'])

def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result
track_features = normalize(track_features)

track_features.head(n=3)

,duration,release_year,us_popularity_estimate,acousticness,beat_strength,bounciness,danceability,dyn_range_mean,energy,flatness,instrumentalness,key,liveness,loudness,mechanism,organism,speechiness,tempo,time_signature,valence,acoustic_vector_0,acoustic_vector_1,acoustic_vector_2,acoustic_vector_3,acoustic_vector_4,acoustic_vector_5,acoustic_vector_6,acoustic_vector_7,mode_major,mode_minor
track_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
t_2e8f4b71-8a0b-4b9c-b7d8-fb5208e87f9f,0.167239,0.304348,0.958289,0.719233,0.366513,0.337232,0.440801,0.110348,0.238851,0.865473,6.533884e-01,0.0,0.769258,0.647393,0.199170,0.775905,0.034969,0.401501,0.8,0.223398,0.607061,0.233876,0.745271,0.656751,0.633754,0.653592,0.288896,0.399172,1.0,0.0
t_dae2ec0e-ec7b-4b3e-b60c-4a884d0eccb0,0.066561,0.188406,0.727204,0.843003,0.362229,0.395253,0.508696,0.130109,0.420482,0.856651,3.941564e-09,0.0,0.085844,0.734892,0.357639,0.763381,0.051367,0.565365,0.8,0.484707,0.558104,0.315577,0.644835,0.757328,0.627808,0.403121,0.510257,0.171852,1.0,0.0
t_cf0164dd-1531-4399-bfa6-dec19cd1fedc,0.045423,0.347826,0.962039,0.054904,0.495025,0.597579,0.553525,0.177940,0.842951,0.820145,1.041599e-01,0.0,0.407325,0.764843,0.304721,0.503674,0.073928,0.555589,0.8,0.818449,0.578480,0.460787,0.567144,0.473720,0.719832,0.685979,0.315397,0.422179,1.0,0.0


## Data Joining

In [14]:
# Join track features and logs
data = logs.join(track_features, on='track_id_clean', how='left')
data['session_id'].nunique()

178342

In [15]:
data.head()

,session_id,session_position,session_length,track_id_clean,skip_1,skip_2,skip_3,not_skipped,context_switch,no_pause_before_play,short_pause_before_play,long_pause_before_play,hist_user_behavior_n_seekfwd,hist_user_behavior_n_seekback,hist_user_behavior_is_shuffle,hour_of_day,premium,context_type_catalog,context_type_charts,context_type_editorial_playlist,context_type_personalized_playlist,context_type_radio,context_type_user_collection,hist_user_behavior_reason_start_appload,hist_user_behavior_reason_start_backbtn,hist_user_behavior_reason_start_clickrow,hist_user_behavior_reason_start_endplay,hist_user_behavior_reason_start_fwdbtn,hist_user_behavior_reason_start_playbtn,hist_user_behavior_reason_start_popup,hist_user_behavior_reason_start_remote,hist_user_behavior_reason_start_trackdone,hist_user_behavior_reason_start_trackerror,hist_user_behavior_reason_start_uriopen,hist_user_behavior_reason_end_appload,hist_user_behavior_reason_end_backbtn,hist_user_behavior_reason_end_clickrow,hist_user_behavior_reason_end_endplay,hist_user_behavior_reason_end_fwdbtn,hist_user_behavior_reason_end_logout,hist_user_behavior_reason_end_popup,hist_user_behavior_reason_end_remote,hist_user_behavior_reason_end_trackdone,hist_user_behavior_reason_end_uriopen,duration,release_year,us_popularity_estimate,acousticness,beat_strength,bounciness,danceability,dyn_range_mean,energy,flatness,instrumentalness,key,liveness,loudness,mechanism,organism,speechiness,tempo,time_signature,valence,acoustic_vector_0,acoustic_vector_1,acoustic_vector_2,acoustic_vector_3,acoustic_vector_4,acoustic_vector_5,acoustic_vector_6,acoustic_vector_7,mode_major,mode_minor
0,0_00006f66-33e5-4de7-a324-2d18e439fc1e,1,20,t_0479f24c-27d2-46d6-a00c-7ec928f2b539,False,False,False,True,0,0,0,0,0,0,True,16,True,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0.084783,0.985507,0.996813,0.015915,0.438572,0.480043,0.654555,0.145592,0.553473,0.886287,3.484481e-03,0.090909,0.678553,0.806081,0.546784,0.327509,0.071830,0.536128,0.8,0.152256,0.164759,0.767624,0.726183,0.441198,0.359116,0.730704,0.236833,0.580984,1.0,0.0
1,0_00006f66-33e5-4de7-a324-2d18e439fc1e,2,20,t_9099cd7b-c238-47b7-9381-f23f2c1d1043,False,False,False,True,0,1,0,0,0,0,True,16,True,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0.116834,0.985507,0.989673,0.062072,0.654835,0.745897,0.879322,0.214942,0.726840,0.878262,1.031319e-07,0.636364,0.104322,0.825062,0.824766,0.134194,0.063012,0.520179,0.8,0.337156,0.211725,0.756546,0.778338,0.407780,0.335201,0.765185,0.230647,0.560879,0.0,1.0
2,0_00006f66-33e5-4de7-a324-2d18e439fc1e,3,20,t_fc5df5ba-5396-49a7-8b29-35d0d28249e0,False,False,False,True,0,1,0,0,0,0,True,16,True,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0.113710,0.985507,0.999998,0.355611,0.532180,0.547930,0.681213,0.153304,0.563018,0.881542,2.659045e-08,0.909091,0.135776,0.817156,0.774327,0.303257,0.046729,0.580142,0.8,0.373866,0.198437,0.762347,0.740823,0.404552,0.375268,0.763143,0.166574,0.570818,1.0,0.0
3,0_00006f66-33e5-4de7-a324-2d18e439fc1e,4,20,t_23cff8d6-d874-4b20-83dc-94e450e8aa20,False,False,False,True,0,1,0,0,0,0,True,16,True,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0.078998,0.985507,0.999504,0.772472,0.641786,0.739370,0.866782,0.214539,0.529492,0.850761,6.598412e-06,0.090909,0.103722,0.788291,0.630996,0.616140,0.236906,0.447951,0.8,0.649426,0.215648,0.734013,0.764560,0.409350,0.330305,0.783042,0.231117,0.614465,1.0,0.0
4,0_00006f66-33e5-4de7-a324-2d18e439fc1e,5,20,t_64f3743c-f624-46bb-a579-0f3f9a07a123,False,False,False,True,0,1,0,0,0,0,True,16,True,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0.102003,0.985507,0.999850,0.006630,0.732462,0.805940,0.859664,0.231531,0.650067,0.856799,2.066657e-06,0.727273,0.120842,0.831098,0.759465,0.173777,0.248284,0.588155,0.8,0.652927,0.140517,0.740710,0.713375,0.468176,0.359157,0.702825,0.154084,0.571117,1.0,0.0


In [16]:
len(data)

2990609

In [22]:
data['premium'] = data['premium']*1
data['hist_user_behavior_is_shuffle'] = data['hist_user_behavior_is_shuffle']*1
data['skip_1'] = data['skip_1']*1
data['skip_2'] = data['skip_2']*1
data['skip_3'] = data['skip_3']*1

In [27]:
data.shape

(2990609, 74)

## Cosine Similarity

In [25]:
data.columns

Index(['session_id', 'session_position', 'session_length', 'track_id_clean',
       'skip_1', 'skip_2', 'skip_3', 'not_skipped', 'context_switch',
       'no_pause_before_play', 'short_pause_before_play',
       'long_pause_before_play', 'hist_user_behavior_n_seekfwd',
       'hist_user_behavior_n_seekback', 'hist_user_behavior_is_shuffle',
       'hour_of_day', 'premium', 'context_type_catalog', 'context_type_charts',
       'context_type_editorial_playlist', 'context_type_personalized_playlist',
       'context_type_radio', 'context_type_user_collection',
       'hist_user_behavior_reason_start_appload',
       'hist_user_behavior_reason_start_backbtn',
       'hist_user_behavior_reason_start_clickrow',
       'hist_user_behavior_reason_start_endplay',
       'hist_user_behavior_reason_start_fwdbtn',
       'hist_user_behavior_reason_start_playbtn',
       'hist_user_behavior_reason_start_popup',
       'hist_user_behavior_reason_start_remote',
       'hist_user_behavior_reason_start

In [29]:
# All songs that have been skipped:
skipped_songs_data = data[data['skip_2'] == 1]

In [34]:
cosine_distance_columns = ['duration', 'release_year', 'us_popularity_estimate', 'acousticness',
       'beat_strength', 'bounciness', 'danceability', 'dyn_range_mean',
       'energy', 'flatness', 'instrumentalness', 'key', 'liveness', 'loudness',
       'mechanism', 'organism', 'speechiness', 'tempo', 'time_signature',
       'valence', 'acoustic_vector_0', 'acoustic_vector_1',
       'acoustic_vector_2', 'acoustic_vector_3', 'acoustic_vector_4',
       'acoustic_vector_5', 'acoustic_vector_6', 'acoustic_vector_7']
skipped_songs_data[cosine_distance_columns]

,duration,release_year,us_popularity_estimate,acousticness,beat_strength,bounciness,danceability,dyn_range_mean,energy,flatness,instrumentalness,key,liveness,loudness,mechanism,organism,speechiness,tempo,time_signature,valence,acoustic_vector_0,acoustic_vector_1,acoustic_vector_2,acoustic_vector_3,acoustic_vector_4,acoustic_vector_5,acoustic_vector_6,acoustic_vector_7
6,0.088391,0.985507,0.986157,0.451527,0.487731,0.617080,0.558746,0.186920,0.469357,0.884593,2.062505e-08,0.545455,0.111306,0.787507,0.157576,0.689625,0.422271,0.417886,0.8,0.109421,0.195737,0.736168,0.730950,0.429520,0.367803,0.742205,0.178184,0.578889
7,0.103981,0.985507,0.724382,0.017703,0.649398,0.692108,0.490481,0.188170,0.715166,0.899285,2.530043e-05,0.090909,0.114787,0.800302,0.354167,0.466589,0.106830,0.332161,0.8,0.389917,0.115941,0.754604,0.767510,0.482714,0.387159,0.749495,0.188923,0.583537
8,0.106362,0.985507,0.999845,0.030254,0.818837,0.892190,0.966503,0.285814,0.555288,0.896443,5.667939e-04,0.454545,0.128244,0.811090,0.857855,0.104936,0.051365,0.520095,0.8,0.338324,0.197577,0.729687,0.726160,0.459495,0.402739,0.737422,0.210639,0.561478
9,0.016949,0.985507,0.995744,0.333870,0.751536,0.855088,0.916091,0.274794,0.239322,0.891250,4.132171e-04,0.363636,0.119036,0.775602,0.622222,0.363435,0.159296,0.400969,0.8,0.257675,0.242272,0.733300,0.739956,0.408108,0.398027,0.760860,0.257891,0.547659
10,0.148433,0.985507,0.999347,0.054664,0.595617,0.652173,0.804592,0.181793,0.639737,0.899758,4.765252e-05,0.363636,0.341846,0.837012,0.851399,0.114292,0.028779,0.527971,0.8,0.037999,0.276153,0.718023,0.739030,0.389392,0.357072,0.799097,0.245211,0.598342
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2990603,0.080957,0.971014,0.999853,0.000859,0.501847,0.588728,0.682970,0.173429,0.658765,0.888341,8.170098e-06,0.818182,0.110393,0.808676,0.702413,0.214916,0.276297,0.624048,0.8,0.224536,0.144006,0.770233,0.701166,0.456703,0.350613,0.687575,0.191576,0.565319
2990604,0.053248,0.971014,0.996565,0.273002,0.480806,0.492447,0.675283,0.142877,0.636059,0.879943,1.623397e-02,0.818182,0.104433,0.817367,0.871094,0.217284,0.041525,0.571986,0.8,0.537848,0.193012,0.716979,0.722059,0.466610,0.384308,0.759385,0.264029,0.614473
2990605,0.080957,0.971014,0.999853,0.000859,0.501847,0.588728,0.682970,0.173429,0.658765,0.888341,8.170098e-06,0.818182,0.110393,0.808676,0.702413,0.214916,0.276297,0.624048,0.8,0.224536,0.144006,0.770233,0.701166,0.456703,0.350613,0.687575,0.191576,0.565319
2990606,0.053248,0.971014,0.996565,0.273002,0.480806,0.492447,0.675283,0.142877,0.636059,0.879943,1.623397e-02,0.818182,0.104433,0.817367,0.871094,0.217284,0.041525,0.571986,0.8,0.537848,0.193012,0.716979,0.722059,0.466610,0.384308,0.759385,0.264029,0.614473


In [44]:
mean_skipped_song = skipped_songs_data[cosine_distance_columns].mean().tolist()

In [45]:
# What variable would be highly correlated or unnecessary?
mean_skipped_song

[0.10558970674108208,
 0.918753077053222,
 0.973245210932833,
 0.21667454474467424,
 0.5467022918317864,
 0.5880484243474116,
 0.6700205421842573,
 0.17288791726879804,
 0.6280399150340111,
 0.8630964026129947,
 0.028308753430010446,
 0.47449613452801154,
 0.18999036636635003,
 0.7996294715485152,
 0.5932863698294962,
 0.35655216612954305,
 0.14899945833463768,
 0.4908557793231019,
 0.7899960611544632,
 0.45807934496653974,
 0.2973591653748634,
 0.7128770787012366,
 0.712327532361058,
 0.42108128973875747,
 0.4488203771063922,
 0.7520581428275951,
 0.2724384445455443,
 0.5217412756776963]

In [53]:
tqdm.pandas()

In [63]:
data['similarity_mean_skipped_song'] = data[cosine_distance_columns].progress_apply(lambda x: cosine_similarity([mean_skipped_song], [x.tolist()])[0][0], axis=1)

100%|██████████| 2990609/2990609 [12:13<00:00, 4074.64it/s]


In [61]:
data['similarity_mean_skipped_song'].min()

0.8639864543014858

In [62]:
data[data['similarity_mean_skipped_song'] == 0.8639864543014858]

,session_id,session_position,session_length,track_id_clean,skip_1,skip_2,skip_3,not_skipped,context_switch,no_pause_before_play,short_pause_before_play,long_pause_before_play,hist_user_behavior_n_seekfwd,hist_user_behavior_n_seekback,hist_user_behavior_is_shuffle,hour_of_day,premium,context_type_catalog,context_type_charts,context_type_editorial_playlist,context_type_personalized_playlist,context_type_radio,context_type_user_collection,hist_user_behavior_reason_start_appload,hist_user_behavior_reason_start_backbtn,hist_user_behavior_reason_start_clickrow,hist_user_behavior_reason_start_endplay,hist_user_behavior_reason_start_fwdbtn,hist_user_behavior_reason_start_playbtn,hist_user_behavior_reason_start_popup,hist_user_behavior_reason_start_remote,hist_user_behavior_reason_start_trackdone,hist_user_behavior_reason_start_trackerror,hist_user_behavior_reason_start_uriopen,hist_user_behavior_reason_end_appload,hist_user_behavior_reason_end_backbtn,hist_user_behavior_reason_end_clickrow,hist_user_behavior_reason_end_endplay,hist_user_behavior_reason_end_fwdbtn,hist_user_behavior_reason_end_logout,hist_user_behavior_reason_end_popup,hist_user_behavior_reason_end_remote,hist_user_behavior_reason_end_trackdone,hist_user_behavior_reason_end_uriopen,duration,release_year,us_popularity_estimate,acousticness,beat_strength,bounciness,danceability,dyn_range_mean,energy,flatness,instrumentalness,key,liveness,loudness,mechanism,organism,speechiness,tempo,time_signature,valence,acoustic_vector_0,acoustic_vector_1,acoustic_vector_2,acoustic_vector_3,acoustic_vector_4,acoustic_vector_5,acoustic_vector_6,acoustic_vector_7,mode_major,mode_minor,similarity_mean_skipped_song
28,0_0000a72b-09ac-412f-b452-9b9e79bded8f,9,20,t_0b38d601-8996-4aec-843c-74f6d96db1f5,1,1,1,False,0,1,0,0,0,0,1,14,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0.483292,0.913043,0.086126,0.06068,0.318643,0.383165,0.368174,0.132359,0.922195,0.778689,0.463558,0.0,0.707743,0.811075,0.192727,0.584636,0.138908,0.630665,0.8,0.605495,0.613512,0.561497,0.479639,0.39382,0.628777,0.782961,0.427332,0.597774,1.0,0.0,0.863986
30,0_0000a72b-09ac-412f-b452-9b9e79bded8f,11,20,t_0b38d601-8996-4aec-843c-74f6d96db1f5,0,0,0,True,0,1,0,0,0,0,1,15,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.483292,0.913043,0.086126,0.06068,0.318643,0.383165,0.368174,0.132359,0.922195,0.778689,0.463558,0.0,0.707743,0.811075,0.192727,0.584636,0.138908,0.630665,0.8,0.605495,0.613512,0.561497,0.479639,0.39382,0.628777,0.782961,0.427332,0.597774,1.0,0.0,0.863986


In [64]:
data.to_csv('/content/drive/My Drive/CS/AI/Data/training_set/log_0_20180715_000000000000_JOINED_FEATURES_COSINE_SIMILARITY.csv')